<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In the following we define the classes [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) and [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) where KDE is short for 'Kernel Density Estimator' and the 'x' is supposed to signal that both classes can be defined based on any arbitrary point predictor. The name 'LevelSet' stems from the fact that every approach presented in this notebook interprets the values of the point forecasts as a similarity measure between samples. The point predictor is specified by the argument `estimator` and must have a `.predict()`-method and should have been trained before hand. 

Both classes [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) and [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) fulfill the same task: By first running `.fit(XTrain, yTrain)` and then calling `.generateWeights(XTest)`, they both output an estimation of the conditional density of every sample specified by 'XTest'. The basic idea for both approaches is also identical: Suppose we have a single test sample at hand. At first, we compare the value of the point prediction of this sample and the values of the point predictions of the training samples computed via `estimator.predict(XTrain)` and `estimator.predict(XTest)`, respectively. Based on this comparison, we select 'binSize'-many training samples that we deem the most similar to the test sample at hand. The concrete way we select the training samples constitutes the only difference between [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) and [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn). Finally, the empirical distribution of the y-values of these training samples then acts as our estimation of the conditional distribution.

Further details on how both approaches work approaches can be found below.

## Level-Set Approach based on Bin Building

In [1]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L33){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx

>      LevelSetKDEx (estimator, binSize:int=100, weightsByDistance:bool=False)

*[`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) turns any point forecasting model into an estimator of the underlying conditional density.
The name 'LevelSet' stems from the fact that this approach interprets the values of the point forecasts
as a similarity measure between samples. The point forecasts of the training samples are sorted and 
recursively assigned to a bin until the size of the current bin reaches `binSize` many samples. Then
a new bin is created and so on. For a new test sample we check into which bin its point prediction
would have fallen and interpret the training samples of that bin as the empirical distribution function
of this test sample.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| estimator |  |  | Model with a .fit and .predict-method (implementing the scikit-learn estimator interface). |
| binSize | int | 100 | Size of the bins created while running fit. |
| weightsByDistance | bool | False | Determines behaviour of method `getWeights`. If False, all weights receive the same  <br>value. If True, the distance of the point forecasts is taking into account. |

In [2]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx.fit)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L73){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx.fit

>      LevelSetKDEx.fit (X:numpy.ndarray, y:numpy.ndarray)

*Fit [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) model by grouping the point predictions of the samples specified via `X`
according to their value. Samples are recursively sorted into bins until each bin contains
`binSize` many samples. For details, checkout the function [`generateBins`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#generatebins) which does the
heavy lifting.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | np.ndarray | Feature matrix used by `estimator` to predict `y`. |
| y | np.ndarray | 1-dimensional target variable corresponding to the feature matrix `X`. |

In [3]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx.getWeights)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L125){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx.getWeights

>      LevelSetKDEx.getWeights (X:numpy.ndarray,
>                               outputType:str='onlyPositiveWeights',
>                               scalingList:list=None)

*Computes estimated conditional density for each sample specified by `X`. The concrete structure of each element 
of the returned list depends on the specified value of `outputType`:

- **all**: An array with the same length as the number of training samples. Each entry represents the probability 
  of each training sample.
- **onlyPositiveWeights**: A tuple. The first element of the tuple represents the probabilities and the second 
  one the indices of the corresponding training sample. Only probalities greater than zero are returned. 
  Note: This is the most memory and computationally efficient output type.
- **summarized**: A tuple. The first element of the tuple represents the probabilities and the second one the 
  corresponding value of `yTrain`. The probabilities corresponding to identical values of `yTrain` are aggregated.
- **cumDistribution**: A tuple. The first element of the tuple represents the probabilities and the second 
  one the corresponding value of `yTrain`.
- **cumDistributionSummarized**: A tuple. The first element of the tuple represents the probabilities and 
  the second one the corresponding value of `yTrain`. The probabilities corresponding to identical values of `yTrain` are aggregated.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | np.ndarray |  | Feature matrix for which conditional density estimates are computed. |
| outputType | str | onlyPositiveWeights | Specifies structure of the returned density estimates. One of: <br>'all', 'onlyPositiveWeights', 'summarized', 'cumDistribution', 'cumDistributionSummarized' |
| scalingList | list | None | Optional. List with length X.shape[0]. Values are multiplied to the estimated <br>density of each sample for scaling purposes. |
| **Returns** | **list** |  | **List whose elements are the conditional density estimates for the samples specified by `X`.** |

#### Generate Bins

In [4]:
#| echo: false
#| output: asis
show_doc(generateBins)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L394){target="_blank" style="float:right; font-size:smaller"}

### generateBins

>      generateBins (binSize:int, yPred:numpy.ndarray)

*Used to generate the bin-structure used by [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) to compute density estimations.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| binSize | int | Size of the bins of values of `yPred` being grouped together. |
| yPred | np.ndarray | 1-dimensional array of predicted values. |

## Level-Set Approach based on DRF

In [ ]:
# #| export

# from drf import drf 

# class LevelSetKDEx_DRF(BaseWeightsBasedEstimator, BaseLSx):
#     """
#     `LevelSetKDEx` turns any point forecasting model into an estimator of the underlying conditional density.
#     The name 'LevelSet' stems from the fact that this approach interprets the values of the point forecasts
#     as a similarity measure between samples. The point forecasts of the training samples are sorted and 
#     recursively assigned to a bin until the size of the current bin reaches `binSize` many samples. Then
#     a new bin is created and so on. For a new test sample we check into which bin its point prediction
#     would have fallen and interpret the training samples of that bin as the empirical distribution function
#     of this test sample.    
#     """
    
#     def __init__(self, 
#                  estimator, # Model with a .fit and .predict-method (implementing the scikit-learn estimator interface).
#                  binSize: int=100, # Size of the bins created while running fit.
#                  ):
        
#         super(BaseEstimator, self).__init__(estimator = estimator)

#         # Check if binSize is integer
#         if not isinstance(binSize, (int, np.int32, np.int64)):
#             raise ValueError("'binSize' must be an integer!")

#         self.binSize = binSize
        
#         self.yTrain = None
#         self.yPredTrain = None
#         self.drf = None
#         self.fitted = False
    
#     #---
    
#     def fit(self: LevelSetKDEx_DRF, 
#             X: np.ndarray, # Feature matrix used by `estimator` to predict `y`.
#             y: np.ndarray, # 1-dimensional target variable corresponding to the feature matrix `X`.
#             ):
#         """
#         Fit `LevelSetKDEx` model by grouping the point predictions of the samples specified via `X`
#         according to their value. Samples are recursively sorted into bins until each bin contains
#         `binSize` many samples. For details, checkout the function `generateBins` which does the
#         heavy lifting.
#         """
        
#         # Checks
#         if not isinstance(self.binSize, (int, np.int32, np.int64)):
#             raise ValueError("'binSize' must be an integer!")
            
#         if self.binSize > y.shape[0]:
#             raise ValueError("'binSize' mustn't be bigger than the size of 'y'!")
        
#         if X.shape[0] != y.shape[0]:
#             raise ValueError("'X' and 'y' must contain the same number of samples!")
        
#         #---
        
#         try:
#             yPred = self.estimator.predict(X)
            
#         except NotFittedError:
#             try:
#                 self.estimator.fit(X = X, y = y)                
#             except:
#                 raise ValueError("Couldn't fit 'estimator' with user specified 'X' and 'y'!")
#             else:
#                 yPred = self.estimator.predict(X)
        
#         #---
        
#         yPred = pd.DataFrame(yPred)
#         y = pd.Series(y)

#         DRF = drf(min_node_size = self.binSize, num_trees = 100, num_features = 1, honesty = False, sample_fraction = 0.5, response_scaling = False, mtry = 1, num_threads = 16)
#         DRF.fit(X = yPred, Y = y)
        
#         #---
        
#         # IMPORTANT: In case 'y' is given as a pandas.Series, we can potentially run into indexing 
#         # problems later on.
#         self.yTrain = y.ravel()
        
#         self.yPredTrain = yPred
#         self.drf = DRF
#         self.fitted = True
        
#     #---
    
#     def getWeights(self: LevelSetKDEx_DRF, 
#                    X: np.ndarray, # Feature matrix for which conditional density estimates are computed.
#                    # Specifies structure of the returned density estimates. One of: 
#                    # 'all', 'onlyPositiveWeights', 'summarized', 'cumDistribution', 'cumDistributionSummarized'
#                    outputType: str='onlyPositiveWeights', 
#                    # Optional. List with length X.shape[0]. Values are multiplied to the estimated 
#                    # density of each sample for scaling purposes.
#                    scalingList: list=None, 
#                    ) -> list: # List whose elements are the conditional density estimates for the samples specified by `X`.
        
#         # __annotations__ = BaseWeightsBasedEstimator.getWeights.__annotations__
#         __doc__ = BaseWeightsBasedEstimator.getWeights.__doc__
        
#         if not self.fitted:
#             raise NotFittedError("This LevelSetKDEx instance is not fitted yet. Call 'fit' with "
#                                  "appropriate arguments before trying to compute weights.")
        
#         #---
        
#         yPred = self.estimator.predict(X)
#         yPred = pd.DataFrame(yPred)
        
#         weightsArray = self.drf.predict(yPred).weights
#         weightsList = list(weightsArray)
#         weightsDataList = [(weights[weights > 0], np.where(weights > 0)[0]) for weights in weightsList]

#         weightsDataList = restructureWeightsDataList(weightsDataList = weightsDataList, 
#                                                      outputType = outputType, 
#                                                      y = self.yTrain,
#                                                      scalingList = scalingList,
#                                                      equalWeights = True)
        
#         return weightsDataList

## Level-Set Approach based on Gaussian Kernel

In [5]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx_RBF)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L435){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx_RBF

>      LevelSetKDEx_RBF (estimator, lengthScale:float=1)

*[`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) turns any point forecasting model into an estimator of the underlying conditional density.
The name 'LevelSet' stems from the fact that this approach interprets the values of the point forecasts
as a similarity measure between samples. The point forecasts of the training samples are sorted and 
recursively assigned to a bin until the size of the current bin reaches `binSize` many samples. Then
a new bin is created and so on. For a new test sample we check into which bin its point prediction
would have fallen and interpret the training samples of that bin as the empirical distribution function
of this test sample.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| estimator |  |  | Model with a .fit and .predict-method (implementing the scikit-learn estimator interface). |
| lengthScale | float | 1 | Size of the bins created while running fit. |

## Level-Set Approach based on kNN

In [6]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx_kNN)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L540){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx_kNN

>      LevelSetKDEx_kNN (estimator, binSize:int=100,
>                        weightsByDistance:bool=False)

*[`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) turns any point predictor that has a .predict-method 
into an estimator of the condititional density of the underlying distribution.
The basic idea of each level-set based approach is to interprete the point forecast
generated by the underlying point predictor as a similarity measure of samples.
In the case of the [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) defined here, for every new samples
'binSize'-many training samples are computed whose point forecast is closest
to the point forecast of the new sample.
The resulting empirical distribution of these 'nearest' training samples are 
viewed as our estimation of the conditional distribution of each the new sample 
at hand.

NOTE: In contrast to the standard [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex), it is possible to apply
[`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) to arbitrary dimensional point predictors.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| estimator |  |  | Model with a .fit and .predict-method (implementing the scikit-learn estimator interface). |
| binSize | int | 100 | Size of the bins created while running fit. |
| weightsByDistance | bool | False | Determines behaviour of method `getWeights`. If False, all weights receive the same  <br>value. If True, the distance of the point forecasts is taking into account. |

In [7]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx_kNN.fit)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L585){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx_kNN.fit

>      LevelSetKDEx_kNN.fit (X:numpy.ndarray, y:numpy.ndarray)

*Fit [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) model by applying the nearest neighbors algorithm to the point
predictions of the samples specified by `X` based on `estimator`.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | np.ndarray | Feature matrix used by `estimator` to predict `y`. |
| y | np.ndarray | 1-dimensional target variable corresponding to the feature matrix `X`. |

In [8]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx_kNN.getWeights)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L640){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx_kNN.getWeights

>      LevelSetKDEx_kNN.getWeights (X:numpy.ndarray,
>                                   outputType:str='onlyPositiveWeights',
>                                   scalingList:list=None)

*Computes estimated conditional density for each sample specified by `X`. The concrete structure of each element 
of the returned list depends on the specified value of `outputType`:

- **all**: An array with the same length as the number of training samples. Each entry represents the probability 
  of each training sample.
- **onlyPositiveWeights**: A tuple. The first element of the tuple represents the probabilities and the second 
  one the indices of the corresponding training sample. Only probalities greater than zero are returned. 
  Note: This is the most memory and computationally efficient output type.
- **summarized**: A tuple. The first element of the tuple represents the probabilities and the second one the 
  corresponding value of `yTrain`. The probabilities corresponding to identical values of `yTrain` are aggregated.
- **cumDistribution**: A tuple. The first element of the tuple represents the probabilities and the second 
  one the corresponding value of `yTrain`.
- **cumDistributionSummarized**: A tuple. The first element of the tuple represents the probabilities and 
  the second one the corresponding value of `yTrain`. The probabilities corresponding to identical values of `yTrain` are aggregated.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | np.ndarray |  | Feature matrix for which conditional density estimates are computed. |
| outputType | str | onlyPositiveWeights | Specifies structure of the returned density estimates. One of: <br>'all', 'onlyPositiveWeights', 'summarized', 'cumDistribution', 'cumDistributionSummarized' |
| scalingList | list | None | Optional. List with length X.shape[0]. Values are multiplied to the estimated <br>density of each sample for scaling purposes. |
| **Returns** | **list** |  | **List whose elements are the conditional density estimates for the samples specified by `X`.** |

## Level-Set Approach based on NN

In [9]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx_NN)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L727){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx_NN

>      LevelSetKDEx_NN (estimator, binSize:int=100, efficientRAM:bool=False)

*[`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) turns any point predictor that has a .predict-method 
into an estimator of the condititional density of the underlying distribution.
The basic idea of each level-set based approach is to interprete the point forecast
generated by the underlying point predictor as a similarity measure of samples.
In the case of the [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) defined here, for every new samples
'binSize'-many training samples are computed whose point forecast is closest
to the point forecast of the new sample.
The resulting empirical distribution of these 'nearest' training samples are 
viewed as our estimation of the conditional distribution of each the new sample 
at hand.

NOTE: In contrast to the standard [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex), it is possible to apply
[`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex_knn) to arbitrary dimensional point predictors.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| estimator |  |  | Model with a .fit and .predict-method (implementing the scikit-learn estimator interface). |
| binSize | int | 100 | Size of the bins created while running fit. |
| efficientRAM | bool | False | Setting 'efficientRAM = TRUE' is only necessary when there are roughly umore than 200k training observations to avoid<br>an overusage of RAM. This setting causes the run-time of the algorithm of the weights computation to linearly depend on <br>'binSize'. Because of that the algorithm becomes quite slow for 'binSize' > 10k'. |

### Get Neighbors

In [10]:
#| echo: false
#| output: asis
show_doc(getNeighbors)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L870){target="_blank" style="float:right; font-size:smaller"}

### getNeighbors

>      getNeighbors (binSize:int, yPred:numpy.ndarray)

*Used to generate the neighboorhoods used by [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) to compute density estimations.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| binSize | int | Size of the bins of values of `yPred` being grouped together. |
| yPred | np.ndarray | 1-dimensional array of predicted values. |

### Get Neighbor Test

In [11]:
#| echo: false
#| output: asis
show_doc(getNeighborsTest)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L1084){target="_blank" style="float:right; font-size:smaller"}

### getNeighborsTest

>      getNeighborsTest (binSize:int, yPred:numpy.ndarray,
>                        yPredTrain:numpy.ndarray, neighborsDictTrain:dict)

*Used to generate the neighboorhoods used by [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) to compute density estimations.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| binSize | int | Size of the bins of values of `yPred` being grouped together. |
| yPred | np.ndarray | 1-dimensional array of predicted values. |
| yPredTrain | np.ndarray | 1-dimensional array of predicted train values. |
| neighborsDictTrain | dict | Dict containing the neighbors of all train samples. Keys are the train predictions. |

### Get Kernel Values

In [12]:
#| echo: false
#| output: asis
show_doc(getKernelValues)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L1185){target="_blank" style="float:right; font-size:smaller"}

### getKernelValues

>      getKernelValues (yPred, yPredTrain, neighborsDictTest,
>                       neighborsDictTrain, neighborsRemoved, neighborsAdded,
>                       binSize, efficientRAM=False)

## Level-Set Approach based on Clustering

In [13]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx_clustering)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L1308){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx_clustering

>      LevelSetKDEx_clustering (estimator, nClusters:int=10)

*[`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) turns any point forecasting model into an estimator of the underlying conditional density.
The name 'LevelSet' stems from the fact that this approach interprets the values of the point forecasts
as a similarity measure between samples. The point forecasts of the training samples are sorted and 
recursively assigned to a bin until the size of the current bin reaches `binSize` many samples. Then
a new bin is created and so on. For a new test sample we check into which bin its point prediction
would have fallen and interpret the training samples of that bin as the empirical distribution function
of this test sample.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| estimator |  |  | Model with a .fit and .predict-method (implementing the scikit-learn estimator interface). |
| nClusters | int | 10 | Number of clusters to form as well as number of centroids to generate. |

In [14]:
#| echo: false
#| output: asis
show_doc(LevelSetKDEx_clustering2)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/levelSetKDEx_univariate.py#L1464){target="_blank" style="float:right; font-size:smaller"}

### LevelSetKDEx_clustering2

>      LevelSetKDEx_clustering2 (estimator, nClusters:int=10)

*[`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex_univariate.html#levelsetkdex) turns any point forecasting model into an estimator of the underlying conditional density.
The name 'LevelSet' stems from the fact that this approach interprets the values of the point forecasts
as a similarity measure between samples. The point forecasts of the training samples are sorted and 
recursively assigned to a bin until the size of the current bin reaches `binSize` many samples. Then
a new bin is created and so on. For a new test sample we check into which bin its point prediction
would have fallen and interpret the training samples of that bin as the empirical distribution function
of this test sample.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| estimator |  |  | Model with a .fit and .predict-method (implementing the scikit-learn estimator interface). |
| nClusters | int | 10 | Number of clusters to form as well as number of centroids to generate. |

# Test Code

In [ ]:
# #| hide

# data, XTrain, yTrain, XTest, yTest = loadDataBakery(returnXY = True)

# LGBM = LGBMRegressor(n_jobs = 1).fit(XTrain, yTrain)

In [ ]:
# #| hide

# LSKDEx_drf = LevelSetKDEx_DRF(estimator = LGBM, binSize = 100)
# LSKDEx_drf.fit(XTrain, yTrain)

# weightsDataList = LSKDEx_drf.getWeights(XTest)

In [ ]:
# #| hide

# import time
# start = time.time()
# LSKDEx = LevelSetKDEx(estimator = LGBM, binSize = 100)

# LSKDEx.fit(XTrain, yTrain)
# weights = LSKDEx.getWeights(XTest)
# print(time.time() - start)

# LSx based on DRF

In [ ]:
# #| hide

# import ipdb
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor
# from dddex.loadData import *
# from dddex.wSAA import RandomForestWSAA, SampleAverageApproximation
# import time 
# import psutil
# import os
# import sys

# from drf import drf

In [ ]:
# data, XTrain, yTrain, XTest, yTest = loadDataBakery(returnXY = True)
# LGBM = LGBMRegressor(n_jobs = 1).fit(XTrain, yTrain)

# LSKDEx = LevelSetKDEx_RBF(estimator = LGBM, lengthScale = 1)

In [ ]:
# LSKDEx.fit(XTrain, yTrain)

In [ ]:
# weightsData = LSKDEx.getWeights(XTest[100:200], outputType = 'summarized')

In [ ]:
# #| hide

# data, XTrain, yTrain, XTest, yTest = loadDataYaz(returnXY = True)

# LGBM = LGBMRegressor(n_jobs = 1).fit(XTrain, yTrain)
# yPredTrain = LGBM.predict(XTrain)
# yPredTest = LGBM.predict(XTest)

# yPredTrain = pd.DataFrame(yPredTrain)
# yPredTest = pd.DataFrame(yPredTest)
# yTrain = pd.Series(yTrain)

In [ ]:
# #| hide

# DRF = drf(min_node_size = 100, num_trees = 500, num_features = 1, honesty = False, sample_fraction = 0.5, response_scaling = False, mtry = 1, num_threads = 1)

In [ ]:
# #| hide

# DRF.fit(yPredTrain, yTrain)
# weights = DRF.predict(yPredTest).weights

In [ ]:
# #| hide

# # Get statistic of weights of first row#
# weightsRow = weights[0]
# pd.Series(weightsRow[weightsRow > 0]).describe()

count    366.000000
mean       0.002732
std        0.002423
min        0.000020
25%        0.000590
50%        0.001994
75%        0.004667
max        0.009120
dtype: float64

In [ ]:
# import time
# start = time.time()
# LSKDEx = LevelSetKDEx_clustering2(estimator = LGBM, nClusters = 100)

# LSKDEx.fit(XTrain, yTrain)
# weights = LSKDEx.getWeights(XTest)
# print(time.time() - start)

In [ ]:
# path = '/home/kagu/SID/data/dataSID.csv'
# data = pd.read_csv(path)

# ids = data.id.unique()[0:30]
# filtering = [ID in ids for ID in data.id]
# data = data[filtering]

# X = np.array(data.drop(['demand', 'date', 'id', 'label'], axis = 1))
# Y = np.array(data['demand'])

# indicesTrain = data['label'] == 'train'
# indicesTest = data['label'] == 'test'

# XTrain = X[indicesTrain]
# yTrain = Y[indicesTrain]

# XTest = X[indicesTest]
# yTest = Y[indicesTest]

# dataTrain = data[indicesTrain]
# dataTest = data[indicesTest]

# scalingList = dataTest['scalingValue'].tolist()

In [ ]:
# data.shape

In [ ]:
# process = psutil.Process(os.getpid())
# print(f"Memory used by Jupyter notebook: {process.memory_info().rss / 2**20:.2f} MB")

In [ ]:
# LGBM = LGBMRegressor(boosting_type = 'gbdt',
#                      n_jobs = 1)

# LGBM.fit(X = XTrain, y = yTrain)

In [ ]:
# start = time.time()
# LSKDEx = LevelSetKDEx(estimator = LGBM, binSize = 5000)
# LSKDEx.fit(XTrain, yTrain)
# print(time.time() - start)

# yPredTrain = LSKDEx.yPredTrain

In [ ]:
# yPredTrain[LSKDEx.neighborsDictTrain[list(LSKDEx.neighborsDictTrain.keys())[100]]]

In [ ]:
# yPredTrain[LSKDEx.neighborsDictTrain[np.array(list(LSKDEx.neighborsDictTrain.keys()))[-1]]]

In [ ]:
# weights = LSKDEx.getWeights(XTest, efficientRAM = True)

In [ ]:
# res = LSKDEx.predict(XTest,
#                      probs = [0.1, 0.5], 
#                      scalingList = scalingList)

In [ ]:
# process = psutil.Process(os.getpid())
# print(f"Memory used by Jupyter notebook: {process.memory_info().rss / 2**20:.2f} MB")

In [ ]:
# XTrainMod = XTrain[0:10000]
# yTrainMod = yTrain[0:10000]
# XTestMod = XTest[0:10]

# yPredTrainMod = LGBM.predict(XTrainMod)
# yPredTestMod = LGBM.predict(XTestMod)

In [ ]:
# #| hide

# LSKDEx = LevelSetKDEx(estimator = LGBM, binSize = 100, weightsByDistance = False)
# LSKDEx.fit(XTrainMod, yTrainMod)

In [ ]:
# %%timeit
# #| hide

# res = LSKDEx.solveKernelGLS(X = XTrainMod, sigma = 0.5, c = yPredTrainMod)

In [ ]:
# %%timeit
# #| hide
# res = LSKDEx.getKernelVectorProduct(X1 = XTrainMod, c = yPredTrainMod)

In [ ]:
# %%timeit

# mean, cov = LSKDEx.getGaussianPosterior(XTrain = XTrainMod, 
#                                         XTest = XTestMod,
#                                         yTrain = yTrainMod,
#                                         sigma = 0.5)

In [ ]:
# yPredTest = LSKDEx.estimator.predict(XTest)
# binPerPredTest = np.searchsorted(a = LSKDEx.lowerBoundPerBin, v = yPredTest, side = 'right') - 1

# binVectorsTest = [(binPerPredTest == i).reshape(-1, 1) * 1 for i in range(len(LSKDEx.lowerBoundPerBin))]
# binVectorsToSliceTest = [np.where(binVector)[0] for binVector in binVectorsTest]

In [ ]:
# v = binVectorsToSliceTest[2]
# cov[v[:, None],  v]

In [ ]:
# v

In [ ]:
# pd.Series(np.ravel(mean)).describe()

In [ ]:
# pd.Series(yTest).describe()

In [ ]:
# yPred = np.concatenate([np.arange(5000)] * 2, axis = 0)
# yPredTrain = np.concatenate([np.arange(50000)] * 2, axis = 0)
# binSize = 200

# neighborsDictTrain, neighborsRemoved, neighborsAdded = generateNeighborhoodsUnique(binSize = binSize,
#                                                                                yPred = yPredTrain)

# neighborsDictTest = generateNeighborhoodsTestUnique(binSize = binSize,
#                                                 yPred = yPred,
#                                                 yPredTrain = yPredTrain,
#                                                 neighborsDictTrain = neighborsDictTrain)

# start = time.time()
# kernelValuesList = getKernelValues(binSize = binSize,
#                                    yPred = yPred,
#                                    yPredTrain = yPredTrain,
#                                    neighborsDictTest = neighborsDictTest,
#                                    neighborsDictTrain = neighborsDictTrain,
#                                    neighborsRemoved = neighborsRemoved,
#                                    neighborsAdded = neighborsAdded)
# print(time.time() - start)